In [1]:
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.stats import norm

sys.path.insert(0, 'C:/Users/Bin/Desktop/Thesis/code/src_3sigma/Initialization')
from EncDecAD import EncDecAD
from sklearn.preprocessing import MinMaxScaler


# HTTP

In [2]:
class Data_Helper(object):
    
    def __init__(self,path,step_num,batch_num):
        self.path = path
        self.step_num = step_num
        self.batch_num = batch_num
       
        self.df = pd.read_csv(self.path[0])
        test_n_data = pd.read_csv(self.path[1])
        test_a_data = pd.read_csv(self.path[2])
        
        print("Preprocessing...")
        
        self.trainingset,_ = self.preprocessing(self.df)
        self.test_n,_ = self.preprocessing(test_n_data)
        _,self.test_a = self.preprocessing(test_a_data)
    
        tt = self.trainingset.shape[0] // step_num
        t1 = self.test_n.shape[0] // step_num
        t2 = self.test_a.shape[0] // step_num
        self.traininglist = [self.trainingset[step_num*i:step_num*(i+1)].as_matrix() for i in range(tt)]
        self.test_n_list = [self.test_n[step_num*i:step_num*(i+1)].as_matrix() for i in range(t1)]
        self.test_a_list = [self.test_a[step_num*i:step_num*(i+1)].as_matrix() for i in range(t2)]
        print("Ready for training.")
  
    
    def preprocessing(self,df):
        
        label = df.iloc[:,-1]
        scaler = MinMaxScaler()
        scaler.fit(df.iloc[:,:-1])
        cont = scaler.transform(df.iloc[:,:-1])
            
        cont = pd.DataFrame(cont)
#        cont.columns = continuous.columns.values
        data = pd.concat((cont,label),axis=1)
        
        # split data according to window length
        n_list = []
        a_list = []
        #new version: iter windows
        windows = [data.iloc[w*self.step_num:(w+1)*self.step_num,:] for w in range(data.index.size//self.step_num)]
        for win in windows:
            label = win.iloc[:,-1]
            if label[label!="normal"].size == 0:
                n_list += [i for i in win.index]
            else:
                a_list += [i for i in win.index]
                
        normal = data.iloc[np.array(n_list),:-1]
        anomaly = data.iloc[np.array(a_list),:-1]

        return normal,anomaly

In [ ]:
results = []

for hidden_num in range(5,55,5):
    for step_num in range(10,110,10):
        tf.reset_default_graph()
        training_data_source='file'

        batch_num = 1
        elem_num = 34

        iteration = 300
        path = ["C:/Users/Bin/Desktop/Thesis/dataset/http_gridsearch_train.csv","C:/Users/Bin/Desktop/Thesis/dataset/http_gridsearch_test_n.csv","C:/Users/Bin/Desktop/Thesis/dataset/http_gridsearch_test_a.csv"]
        data_helper = Data_Helper(path,step_num,batch_num)
        train_list = data_helper.traininglist 
        test_n_list = data_helper.test_n_list
        test_a_list = data_helper.test_a_list

        #************#
        # Training
        #************#

        p_input = tf.placeholder(tf.float32, shape=(batch_num, step_num, elem_num),name = "p_input")
        p_inputs = [tf.squeeze(t, [1]) for t in tf.split(p_input, step_num, 1)]

        p_is_training = tf.placeholder(tf.bool,name= "is_training_")

    
        ae = EncDecAD(hidden_num, p_inputs, p_is_training , decode_without_input=False)
        print("Training start.")
        
        with tf.Session() as sess:
            errorBuf = []
            
            graph = tf.get_default_graph()
            sess.run(tf.global_variables_initializer())
            input_= tf.transpose(tf.stack(p_inputs), [1, 0, 2])    
            output_ = graph.get_tensor_by_name("decoder/output_:0")
            print("Training...")
            for i in range(iteration):
                data =[]
                for temp in range(batch_num):
                    ind = np.random.randint(0,len(train_list))
                    sub = train_list[ind]
                    data.append(sub)
                data = np.array(data)
                (ein,aus,loss_val, _) = sess.run([input_,output_,ae.loss, ae.train], {p_input: data,p_is_training : True})
                abs_err = abs(ein-aus)
                for e in abs_err:
                    errorBuf.append(e)
            train_mean = np.array(errorBuf).ravel().mean()
            
            print("testing...")
            error_n_buf = []
            for _ in range(len(test_n_list)):
                data = []
                for _ in range(batch_num):
                    ind = np.random.randint(0,len(test_n_list)-1)
                    sub = test_n_list[ind]
                    data.append(sub)
                    data = np.array(data)
                    (input_n, output_n) = sess.run([input_, output_], {p_input: data, p_is_training: False})
                    err_n = abs(input_n-output_n)
                    for e in err_n:
                            error_n_buf.append(e)
            test_n_mean = np.array(error_n_buf).ravel().mean()
            
            error_a_buf = []
            for _ in range(len(test_a_list)):
                data = []
                for _ in range(batch_num):
                    ind = np.random.randint(0,len(test_a_list))
                    sub = test_a_list[ind]
                    data.append(sub)
                    data = np.array(data)
                    (input_n, output_n) = sess.run([input_, output_], {p_input: data, p_is_training: False})
                    err_n = abs(input_n-output_n)
                    for e in err_n:
                            error_a_buf.append(e)
            test_a_mean = np.array(error_a_buf).ravel().mean()
            results.append([hidden_num,step_num,train_mean,test_n_mean,test_a_mean,test_a_mean/test_n_mean])
            f= open(b"C:\Users\Bin\Desktop\Thesis\code\src\GridSearch\http_results.txt","a")
            f.write(str([hidden_num,step_num,train_mean,test_n_mean,test_a_mean,test_a_mean/test_n_mean])+"\n")
            f.close()
            sess.close()

# SMTP

In [ ]:
results = []

for hidden_num in range(5,55,5):
    for step_num in range(10,110,10):
        tf.reset_default_graph()
        training_data_source='file'

        batch_num = 1
        elem_num = 34

        iteration = 300
        path = ["C:/Users/Bin/Desktop/Thesis/dataset/smtp_gridsearch_train.csv","C:/Users/Bin/Desktop/Thesis/dataset/smtp_gridsearch_test_n.csv","C:/Users/Bin/Desktop/Thesis/dataset/smtp_gridsearch_test_a.csv"]
        data_helper = Data_Helper(path,step_num,batch_num)
        train_list = data_helper.traininglist 
        test_n_list = data_helper.test_n_list
        test_a_list = data_helper.test_a_list

        #************#
        # Training
        #************#

        p_input = tf.placeholder(tf.float32, shape=(batch_num, step_num, elem_num),name = "p_input")
        p_inputs = [tf.squeeze(t, [1]) for t in tf.split(p_input, step_num, 1)]

        p_is_training = tf.placeholder(tf.bool,name= "is_training_")

    
        ae = EncDecAD(hidden_num, p_inputs, p_is_training , decode_without_input=False)
        print("Training start.")
        
        with tf.Session() as sess:
            errorBuf = []
            
            graph = tf.get_default_graph()
            sess.run(tf.global_variables_initializer())
            input_= tf.transpose(tf.stack(p_inputs), [1, 0, 2])    
            output_ = graph.get_tensor_by_name("decoder/output_:0")
            print("Training...")
            for i in range(iteration):
                data =[]
                for temp in range(batch_num):
                    ind = np.random.randint(0,len(train_list))
                    sub = train_list[ind]
                    data.append(sub)
                data = np.array(data)
                (ein,aus,loss_val, _) = sess.run([input_,output_,ae.loss, ae.train], {p_input: data,p_is_training : True})
                abs_err = abs(ein-aus)
                for e in abs_err:
                    errorBuf.append(e)
            train_mean = np.array(errorBuf).ravel().mean()
            
            print("testing...")
            error_n_buf = []
            for _ in range(len(test_n_list)):
                data = []
                for _ in range(batch_num):
                    ind = np.random.randint(0,len(test_n_list))
                    sub = test_n_list[ind]
                    data.append(sub)
                    data = np.array(data)
                    (input_n, output_n) = sess.run([input_, output_], {p_input: data, p_is_training: False})
                    err_n = abs(input_n-output_n)
                    for e in err_n:
                            error_n_buf.append(e)
            test_n_mean = np.array(error_n_buf).ravel().mean()
            
            error_a_buf = []
            for _ in range(len(test_a_list)):
                data = []
                for _ in range(batch_num):
                    ind = np.random.randint(0,len(test_a_list))
                    sub = test_a_list[ind]
                    data.append(sub)
                    data = np.array(data)
                    (input_n, output_n) = sess.run([input_, output_], {p_input: data, p_is_training: False})
                    err_n = abs(input_n-output_n)
                    for e in err_n:
                            error_a_buf.append(e)
            test_a_mean = np.array(error_a_buf).ravel().mean()
            results.append([hidden_num,step_num,train_mean,test_n_mean,test_a_mean,test_a_mean/test_n_mean])
            f= open(b"C:\Users\Bin\Desktop\Thesis\code\src\GridSearch\smtp_results.txt","a")
            f.write(str([hidden_num,step_num,train_mean,test_n_mean,test_a_mean,test_a_mean/test_n_mean])+"\n")
            f.close()
            sess.close()

# Forest

In [ ]:
results = []

for hidden_num in range(5,55,5):
    for step_num in range(10,110,10):
        tf.reset_default_graph()
        training_data_source='file'

        batch_num = 1
        elem_num = 7

        iteration = 300
        path = ["C:/Users/Bin/Desktop/Thesis/dataset/forest_gridsearch_train.csv","C:/Users/Bin/Desktop/Thesis/dataset/forest_gridsearch_test_n.csv","C:/Users/Bin/Desktop/Thesis/dataset/forest_gridsearch_test_a.csv"]
        data_helper = Data_Helper(path,step_num,batch_num)
        train_list = data_helper.traininglist 
        test_n_list = data_helper.test_n_list
        test_a_list = data_helper.test_a_list

        #************#
        # Training
        #************#

        p_input = tf.placeholder(tf.float32, shape=(batch_num, step_num, elem_num),name = "p_input")
        p_inputs = [tf.squeeze(t, [1]) for t in tf.split(p_input, step_num, 1)]

        p_is_training = tf.placeholder(tf.bool,name= "is_training_")

    
        ae = EncDecAD(hidden_num, p_inputs, p_is_training , decode_without_input=False)
        print("Training start.")
        
        with tf.Session() as sess:
            errorBuf = []
            
            graph = tf.get_default_graph()
            sess.run(tf.global_variables_initializer())
            input_= tf.transpose(tf.stack(p_inputs), [1, 0, 2])    
            output_ = graph.get_tensor_by_name("decoder/output_:0")
            print("Training...")
            for i in range(iteration):
                data =[]
                for temp in range(batch_num):
                    ind = np.random.randint(0,len(train_list))
                    sub = train_list[ind]
                    data.append(sub)
                data = np.array(data)
                (ein,aus,loss_val, _) = sess.run([input_,output_,ae.loss, ae.train], {p_input: data,p_is_training : True})
                abs_err = abs(ein-aus)
                for e in abs_err:
                    errorBuf.append(e)
            train_mean = np.array(errorBuf).ravel().mean()
            
            print("testing...")
            error_n_buf = []
            for _ in range(len(test_n_list)):
                data = []
                for _ in range(batch_num):
                    ind = np.random.randint(0,len(test_n_list))
                    sub = test_n_list[ind]
                    data.append(sub)
                    data = np.array(data)
                    (input_n, output_n) = sess.run([input_, output_], {p_input: data, p_is_training: False})
                    err_n = abs(input_n-output_n)
                    for e in err_n:
                            error_n_buf.append(e)
            test_n_mean = np.array(error_n_buf).ravel().mean()
            
            error_a_buf = []
            for _ in range(len(test_a_list)):
                data = []
                for _ in range(batch_num):
                    ind = np.random.randint(0,len(test_a_list))
                    sub = test_a_list[ind]
                    data.append(sub)
                    data = np.array(data)
                    (input_n, output_n) = sess.run([input_, output_], {p_input: data, p_is_training: False})
                    err_n = abs(input_n-output_n)
                    for e in err_n:
                            error_a_buf.append(e)
            test_a_mean = np.array(error_a_buf).ravel().mean()
            results.append([hidden_num,step_num,train_mean,test_n_mean,test_a_mean,test_a_mean/test_n_mean])
            f= open("C:/Users/Bin/Desktop/Thesis/code/src/GridSearch/forest_results.txt","a")
            f.write(str([hidden_num,step_num,train_mean,test_n_mean,test_a_mean,test_a_mean/test_n_mean])+"\n")
            f.close()
            sess.close()

# Power

In [ ]:
results = []

for hidden_num in range(5,55,5):
    for step_num in range(10,110,10):
        tf.reset_default_graph()
        training_data_source='file'

        batch_num = 1
        elem_num = 1

        iteration = 300
        path = ["C:/Users/Bin/Desktop/Thesis/dataset/power_gridsearch_train.csv","C:/Users/Bin/Desktop/Thesis/dataset/power_gridsearch_test_n.csv","C:/Users/Bin/Desktop/Thesis/dataset/power_gridsearch_test_a.csv"]
        data_helper = Data_Helper(path,step_num,batch_num)
        train_list = data_helper.traininglist 
        test_n_list = data_helper.test_n_list
        test_a_list = data_helper.test_a_list

        #************#
        # Training
        #************#

        p_input = tf.placeholder(tf.float32, shape=(batch_num, step_num, elem_num),name = "p_input")
        p_inputs = [tf.squeeze(t, [1]) for t in tf.split(p_input, step_num, 1)]

        p_is_training = tf.placeholder(tf.bool,name= "is_training_")

    
        ae = EncDecAD(hidden_num, p_inputs, p_is_training , decode_without_input=False)
        print("Training start.")
        
        with tf.Session() as sess:
            errorBuf = []
            
            graph = tf.get_default_graph()
            sess.run(tf.global_variables_initializer())
            input_= tf.transpose(tf.stack(p_inputs), [1, 0, 2])    
            output_ = graph.get_tensor_by_name("decoder/output_:0")
            print("Training...")
            for i in range(iteration):
                data =[]
                for temp in range(batch_num):
                    ind = np.random.randint(0,len(train_list)-1)
                    sub = train_list[ind]
                    data.append(sub)
                data = np.array(data)
                (ein,aus,loss_val, _) = sess.run([input_,output_,ae.loss, ae.train], {p_input: data,p_is_training : True})
                abs_err = abs(ein-aus)
                for e in abs_err:
                    errorBuf.append(e)
            train_mean = np.array(errorBuf).ravel().mean()
            
            print("testing...")
            error_n_buf = []
            for _ in range(len(test_n_list)):
                data = []
                for _ in range(batch_num):
                    ind = np.random.randint(0,len(test_n_list))
                    sub = test_n_list[ind]
                    data.append(sub)
                    data = np.array(data)
                    (input_n, output_n) = sess.run([input_, output_], {p_input: data, p_is_training: False})
                    err_n = abs(input_n-output_n)
                    for e in err_n:
                            error_n_buf.append(e)
            test_n_mean = np.array(error_n_buf).ravel().mean()
            
            error_a_buf = []
            for _ in range(len(test_a_list)):
                data = []
                for _ in range(batch_num):
                    ind = np.random.randint(0,len(test_a_list))
                    sub = test_a_list[ind]
                    data.append(sub)
                    data = np.array(data)
                    (input_n, output_n) = sess.run([input_, output_], {p_input: data, p_is_training: False})
                    err_n = abs(input_n-output_n)
                    for e in err_n:
                            error_a_buf.append(e)
            test_a_mean = np.array(error_a_buf).ravel().mean()
            results.append([hidden_num,step_num,train_mean,test_n_mean,test_a_mean,test_a_mean/test_n_mean])
            f= open("C:/Users/Bin/Desktop/Thesis/code/src/GridSearch/power_results.txt","a")
            f.write(str([hidden_num,step_num,train_mean,test_n_mean,test_a_mean,test_a_mean/test_n_mean])+"\n")
            f.close()
            sess.close()

Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning: Mean of empty slice.
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: RuntimeWarning: Mean of empty slice.


Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training start.
Training...
testing...
Preprocessing...
Ready for training.
Training sta